In [1]:
import warnings

warnings.filterwarnings("ignore")

from typing import List, Optional, Union

import numpy as np
import pandas as pd

from tsururu.dataset import IndexSlicer, Pipeline, TSDataset
from tsururu.models import CatBoostRegressor_CV
from tsururu.strategies import (
    DirectStrategy,
    FlatWideMIMOStrategy,
    MIMOStrategy,
    RecursiveStrategy,
)
from tsururu.transformers import (
    DateSeasonsGenerator,
    DifferenceNormalizer,
    LagTransformer,
    LastKnownNormalizer,
    SequentialTransformer,
    StandardScalerTransformer,
    TargetGenerator,
    UnionTransformer,
)
from tsururu.models import MeanBlender

In [2]:
def get_results(
    cv: int,
    regime: str,
    y_true: Optional[List[np.ndarray]] = None,
    y_pred: Optional[List[np.ndarray]] = None,
    ids: Optional[List[Union[float, str]]] = None,
) -> pd.DataFrame:
    def _get_fold_value(
        value: Optional[Union[float, np.ndarray]], idx: int
    ) -> List[Optional[Union[float, np.ndarray]]]:
        if value is None:
            return [None]
        if isinstance(value[idx], float):
            return value[idx]
        if isinstance(value[idx], np.ndarray):
            return value[idx].reshape(-1)
        raise TypeError(f"Unexpected value type. Value: {value}")

    df_res_dict = {}

    for idx_fold in range(cv):
        # Fill df_res_dict
        for name, value in [("y_true", y_true), ("y_pred", y_pred)]:
            df_res_dict[f"{name}_{idx_fold+1}"] = _get_fold_value(
                value, idx_fold
            )
        if regime != "local":
            df_res_dict[f"id_{idx_fold+1}"] = _get_fold_value(ids, idx_fold)

    # Save datasets to specified directory
    df_res = pd.DataFrame(df_res_dict)
    return df_res

There are several main objects to look out for when working with the library:
1) `TSDataset`.
2) `Pipeline` and `Transformers`
3) `Strategy`.
4) `Model`.

### TSDataset

This class is needed to store data and meta-information about it.

To initialise it is necessary to submit the data in `pd.DataFrame` format and define some meta-information about roles that necessary for solving the task of time series forecasting: `id`, `date`, `target`.

In [3]:
df_path = "datasets/global/ettm1.csv"

dataset_params = {
    "target": {
        "columns": ["value"],
        "type": "continious",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    }
}

In [4]:
dataset = TSDataset(
    data=pd.read_csv(df_path),
    columns_params=dataset_params,
)

freq: less then Day (Hour, Min, Sec, etc); period: 900.0 seconds


### Pipeline and Transformers

#### What kind of transformers are there?

Special attention should be paid to the `Transformer` class: the elements of the pipeline that are responsible for transforming the values of a series and generating features. `Pipeline` class is a wrapper over transformers which is needed to provide some additional methods and functions above them.

There are two types of transformers that are used to collect pipelines:
- `Union` transformers;
- `Sequential` transformers.

Below is a list of available Transformers: 
- `StandardScalerTransformer` *(Series4Series)*.
- `DifferenceNormalizer` *(Series4Series)*: subtract the previous value or divide by it.
- `LastKnownNormalizer` *(Features4Features)*: normalize all lags by the last known one: divide by it or subtract.

This three transformers provide flags `transform_features` / `transform_target`, that allow you to manipulate traits and targets separately and get different results from them.

Besides, __DifferenceNormalizer__ and __LastKnownNormalizer__ can be applied in two regimes: `delta` and `ratio`: in the first case, normalisation means subtracting the target value from the current value, and in the second, dividing by it.

- `LabelEncodingTransformer` and `OneHotEncodingTransformer` *(Series4Series)* - encoders for categorical features.
- `TimeToNumGenerator` and `DateSeasonsGenerator` *(Series4Series)* - generator for seasonal features by dates.
- `LagTransformer` *(Series4Features) - generator for lags. 

__!!!The lag transformer must necessarily be present in the sequential transformer, otherwise the features will not be generated.!!!__

Finally, to generate targets, you need to use `TargetGenerator`.

#### Transformers must be assembled in order!

The __SeriesToSeries__ transformers should come first, followed by the LagTransformer and TargetGenerator (__SeriesToFeatures__), and then the __FeaturesToFeatures__ transformers.

#### How to build a Pipeline?

So, there are two ways to build a pipline from transformers: initialise the transformers of interest by hand or use a config in the form of a dictionary. Let's look at both ways.

In [5]:
standard_scaler = StandardScalerTransformer(
    transform_features=True,
    transform_target=True
)

lag = LagTransformer(lags=3)
date_lag = LagTransformer(lags=3)
id_lag = LagTransformer(lags=1)

target_generator = TargetGenerator()

date_seasons = DateSeasonsGenerator(
    seasonalities=["doy", "m", "wd"],
    from_target_date=True,
)

In [6]:
union_1 = UnionTransformer(transformers_list=[lag, target_generator])

seq_1 = SequentialTransformer(transformers_list=[standard_scaler, union_1], input_features=["value"])
seq_2 = SequentialTransformer(transformers_list=[date_seasons, date_lag], input_features=["date"])
seq_3 = SequentialTransformer(transformers_list=[id_lag], input_features=["id"])

union = UnionTransformer(transformers_list=[seq_1, seq_2, seq_3])

In [7]:
pipeline_1 = Pipeline(union, multivariate=False)

In [8]:
pipeline_1.__dict__

{'transformers': <tsururu.transformers.base.UnionTransformer at 0x1dc602dbd50>,
 'multivariate': False,
 'is_fitted': False,
 'strategy_name': None,
 'output_features': None,
 'y_original_shape': None}

Or:

In [9]:
pipeline_params = {
    "target": {
        "columns": ["value"],
        "features": {
            "StandardScalerTransformer":
                {
                    "transform_target": True, 
                    "transform_features": True
                },
            "LagTransformer": {"lags": 7},
        },
    },
    "date": {
        "columns": ["date"],
        "features": {
            "DateSeasonsGenerator": {
                # Use seasonality features from the date column as 
                # features with datetime lags
                # Possible values: [
                #    "y": year, "m": month, "d": day, 
                #    "wd": weekday, "doy": dayofyear,
                #    "hour": hour, "min": minute, "sec": second, 
                #    "ms": microsecond,  "ns": nanosecond
                # ]
                "seasonalities": ['doy', 'm', 'wd'], 
                # Use date from target point to make datetime features
                "from_target_date": True,
            },
            "LagTransformer": {"lags": 3}
        },
    },
    "id": {
        "columns": ["id"],
        "features": {
            "LagTransformer": {"lags": 1},
        },
    }
}

In [10]:
pipeline = Pipeline.from_dict(pipeline_params, multivariate=False)

#### Can I use exogenous variables in the pipeline?

Yes! Exogenous variables can also be specified here. Just include them in your pipeline.

However, their operation is currently tested only for the `MIMOStrategy` in global-modelling. For other strategies support of additional variables is under development.

In [11]:
# pipeline_params["exog_group_1"] = {
#     "columns": ["value"],
#     "features": {
#         "StandardScalerTransformer":
#             {
#                 "transform_target": False, 
#                 "transform_features": True
#             },
#         "LagTransformer": {"lags": 7},
#     },
# }

__Make sure you have the transform_target = False flag for exogenous features!__

#### Model

The model is separate from the strategy. Any model can be run in any strategy if it supports this input and output format

One of the easiest options – is to use GBM.

In [12]:
# Configure the model parameters
model_params = {
    "loss_function": "MultiRMSE",
    "early_stopping_rounds": 100,
    "verbose": 500,
}

# Configure the validation parameters
validation_params = {
    "type": 'KFold',
    "n_splits": 2,
}

In [13]:
model = CatBoostRegressor_CV(validation_params, model_params)

# Blender test
blender = MeanBlender()

#### Strategy

- _Recursive:_ 
    - one model for all points of the forecast horizon;
    - *training*: the model is trained to predict one point ahead;
    - *prediction*: a prediction is iteratively made one point ahead, and then this prediction is used to further shape the features in the test data. 
- _Recursive-reduced:_
    - one model for all points in the prediction horizon;
    - *training*: the model is trained to predict one point ahead;
    - *prediction*: features are generated for all test observations at once, unavailable values are replaced by NaN.
- _Direct:_ 
    - individual models for each point in the prediction horizon. 
- _MultiOutput (MIMO - Multi-input-multi-output):_
    - one model that learns to predict the entire prediction horizon. 
- _FlatWideMIMO:_.
    - mixture of Direct and MIMO, fit one model, but uses deployed over horizon Direct's features.

In [14]:
horizon = 3
history = 7
step = 1

strategy = RecursiveStrategy(horizon, history, step, model, pipeline)

# Blender test
strategy2 = DirectStrategy(horizon, history, step, model, pipeline)

In [15]:
fit_time, _ = strategy.fit(dataset)
fit_time2, _ = strategy2.fit(dataset)

0:	learn: 0.9745768	test: 0.9727270	best: 0.9727270 (0)	total: 154ms	remaining: 2m 33s
500:	learn: 0.2140805	test: 0.2164315	best: 0.2164315 (500)	total: 3.38s	remaining: 3.37s
999:	learn: 0.2104164	test: 0.2147511	best: 0.2147511 (999)	total: 6.56s	remaining: 0us

bestTest = 0.2147510735
bestIteration = 999

Fold 0:
MultiRMSE: 0.21475107347532177
0:	learn: 0.9727297	test: 0.9745896	best: 0.9745896 (0)	total: 8.8ms	remaining: 8.79s
500:	learn: 0.2141088	test: 0.2163332	best: 0.2163332 (500)	total: 3.19s	remaining: 3.17s
999:	learn: 0.2101859	test: 0.2146711	best: 0.2146711 (999)	total: 6.39s	remaining: 0us

bestTest = 0.2146710724
bestIteration = 999

Fold 1:
MultiRMSE: 0.21467107242976863
Mean MultiRMSE: 0.2147
Std: 0.0
0:	learn: 0.9745768	test: 0.9727270	best: 0.9727270 (0)	total: 11.9ms	remaining: 11.9s
500:	learn: 0.2140805	test: 0.2164315	best: 0.2164315 (500)	total: 3.21s	remaining: 3.2s
999:	learn: 0.2104164	test: 0.2147511	best: 0.2147511 (999)	total: 6.38s	remaining: 0us

best

In [16]:
forecast_time, current_pred = strategy.predict(dataset)

# Blender test
forecast_time2, current_pred2 = strategy2.predict(dataset)

freq: less then Day (Hour, Min, Sec, etc); period: 900.0 seconds
freq: less then Day (Hour, Min, Sec, etc); period: 900.0 seconds


In [17]:
current_pred.head(5)

,id,date,value
0,0,2018-06-26 20:00:00,12.690904
1,0,2018-06-26 20:15:00,12.70442
2,0,2018-06-26 20:30:00,12.775842
3,1,2018-06-26 20:00:00,3.764904
4,1,2018-06-26 20:15:00,3.749071


In [18]:
current_pred2.head(5)

,id,date,value
0,0,2018-06-26 20:00:00,12.690904
1,0,2018-06-26 20:15:00,12.716451
2,0,2018-06-26 20:30:00,12.728854
3,1,2018-06-26 20:00:00,3.764904
4,1,2018-06-26 20:15:00,3.752192


In [19]:
#final_preds = blender.fit_predict([current_pred.value, current_pred2.value])

## Backtest validation of pipeline

In [ ]:
ids, test, pred, fit_time, forecast_time = strategy.back_test(dataset, cv=1)

# Blender test
ids2, test2, pred2, fit_time2, forecast_time2 = strategy2.back_test(dataset, cv=1)

freq: less then Day (Hour, Min, Sec, etc); period: 900.0 seconds
0:	learn: 0.9755464	test: 0.9717184	best: 0.9717184 (0)	total: 9.93ms	remaining: 9.92s
500:	learn: 0.2147060	test: 0.2157572	best: 0.2157572 (500)	total: 3.22s	remaining: 3.21s
999:	learn: 0.2108800	test: 0.2139760	best: 0.2139760 (999)	total: 6.39s	remaining: 0us

bestTest = 0.2139760047
bestIteration = 999

Fold 0:
MultiRMSE: 0.21397600470413647
0:	learn: 0.9717506	test: 0.9756144	best: 0.9756144 (0)	total: 8.92ms	remaining: 8.91s
500:	learn: 0.2136825	test: 0.2171605	best: 0.2171605 (500)	total: 3.31s	remaining: 3.3s
999:	learn: 0.2098378	test: 0.2154218	best: 0.2154218 (999)	total: 6.58s	remaining: 0us

bestTest = 0.2154217761
bestIteration = 999

Fold 1:
MultiRMSE: 0.21542177614212452
Mean MultiRMSE: 0.2147
Std: 0.0007
freq: less then Day (Hour, Min, Sec, etc); period: 900.0 seconds
freq: less then Day (Hour, Min, Sec, etc); period: 900.0 seconds
0:	learn: 0.9755464	test: 0.9717184	best: 0.9717184 (0)	total: 8.57ms	r

In [ ]:
pred

In [ ]:
pred2

In [ ]:
# Synthetic preds
pred3 = [el/2 for el in pred2]
pred3

In [ ]:
# Mean blender
final_preds = blender.fit_predict([pred, pred2, pred3])
final_preds

In [ ]:
(pred[0]+pred2[0]+pred3[0])/3 == final_preds[0]

In [ ]:
get_results(cv=1, regime="global", y_true=test, y_pred=final_preds, ids=ids)

## Working with raw time series' granularity

Time series come in different granularities, from hourly and daily time series to more complex ones such as the end of each quarter.

If the rows do not contain segments that are too short (that are shorter than history + horizon), then `tsururu` will try to extract the row granularity on its own. We currently support the following types:

- Yearly (and YearlyEnd)
- Quarterly (and Quarterly)
- Monthly (and MonthlyEnd)
- Weekly
- Daily
- Hourly
- Minlutely
- Secondly
- Microsecondly

There is also support for compound granularities (10 days, 15 minutes, 32 seconds, etc.). The correctness of the selected granularity can be checked from the output after the `Dataset` class has been created.

However, there are tricky situations (e.g. 28 days) where the monthly granularity may be guessed incorrectly. Therefore, it is possible to set your own granularity using the `pd.DateOffset` class or related classes from `pandas.tseries.offsets`, which must be fed as `delta` parameter into the `Dataset` class. Then the time column will be processed according to the user's settings.

Consider a time series where each point is exactly __28 daily points away__ from each other

In [ ]:
df_path_2 = "datasets/global/simulated_data_to_check_28D.csv"

# Configure the features settings
dataset_params_2 = {
    "target": {
        "columns": ["value"],
        "type": "continious",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    }
}

In [ ]:
dataset_2 = TSDataset(
    data=pd.read_csv(df_path_2),
    columns_params=dataset_params_2,
)

We see that the frequency of the series is incorrectly defined as monthly. Let's try to pass the `delta` parameter.

In [ ]:
dataset_2 = TSDataset(
    data=pd.read_csv(df_path_2),
    columns_params=dataset_params_2,
    delta=pd.DateOffset(days=28),
)

Now it's all detected correctly.